# Synthesizing the sound of thunder

Work in progress! Still misses some features not yet implemented in `ipytone`.

- paper: https://arxiv.org/abs/2204.08026
- reference implementation: https://github.com/bineferg/thunder-synthesis

In [ ]:
import random
import ipytone
import numpy as np

In [ ]:
def create_panner(x, y, z):
    return ipytone.Panner3D(
        position=(x, y, z),
        orientation=(0, 0, -1),
        rolloff_factor=10,
        cone_inner_angle=60,
        cone_outer_angle=90,
        cone_outer_gain=0.6,
    )

In [ ]:
reverb = ipytone.Reverb(wet=0.3, decay=10)
delay1 = ipytone.PingPongDelay(wet=0.1, delay_time=0.2, feedback=0.4)
delay2 = ipytone.PingPongDelay(wet=0.2, delay_time=2, feedback=0.1)
lp_filter = ipytone.Filter(frequency=14000, q=0)
reverb.chain(lp_filter, delay1, delay2, ipytone.get_destination())

## Strikes

In [ ]:
class Strike:
    def __init__(self):
        self.noise = ipytone.Noise().start()
        self.envelopes = [
            ipytone.AmplitudeEnvelope(attack=0, decay=0, release=0) for _ in range(4)
        ]
        self.filters = [
            ipytone.Filter(frequency=1000, type="bandpass", q=7)
            for _ in range(8)
        ]
        self.out = ipytone.Gain(gain=1)
        self.delay = ipytone.FeedbackDelay(wet=0.1, delay_time=0.6, feedback=0.15)
        self.panner = create_panner(
            random.uniform(-1, 1),
            random.uniform(-0.5, 0.5),
            random.uniform(-0.5, 0.5),
        )
        
        self._connect()
    
    def _connect(self):
        for i in range(4):
            a = i * 2
            b = i * 2 + 1

            self.noise.chain(self.envelopes[i])
            self.envelopes[i].fan(self.filters[a], self.filters[b])
            self.filters[a].connect(self.out)
            self.filters[b].connect(self.out)

        self.out.chain(self.delay, self.panner)
        
    @property
    def output(self):
        return self.panner
        
    def strike(self, delay):
        for i in range(4):
            r = random.random()
            duration = 140 * (1.4 - r)**5 / 1000
            freq = r * 1200 + 100

            self.envelopes[i].trigger_attack_release(
                duration, f"+{delay}"
            )
            self.filters[i * 2].frequency.set_value_at_time(
                freq, f"+{delay} - 0.001"
            )
            self.filters[i * 2 + 1].frequency.set_value_at_time(
                freq * 0.5, f"+{delay} - 0.001"
            )

    def dispose(self):
        self.noise.dispose()
        self.out.dispose()
        self.delay.dispose()
        self.panner.dispose()
        for env in self.envelopes:
            env.dispose()
        for filtr in self.filters:
            filtr.dispose()

In [ ]:
strikes = []
for i in range(4):
    strikes.append(Strike())
    strikes[i].output.connect(reverb)

In [ ]:
for strike in strikes:
    strike.strike(0.29 + random.uniform(0, 0.25))

## After image

In [ ]:
class AfterImage:
    def __init__(self):
        self.noise1 = ipytone.Noise().start()
        self.noise2 = ipytone.Noise().start()
        self.gain = ipytone.Gain(gain=0)
        self.lp_filter = ipytone.Filter(frequency=33, q=3)
        self.bp_filter = ipytone.Filter(type="bandpass", frequency=333, q=4)
        self.panner = create_panner(
            random.uniform(-1, 1),
            random.uniform(-0.5, 0.5),
            random.uniform(-0.5, 0.5),
        )
        
        self._connect()
    
    def _connect(self):
        self.mult = ipytone.Multiply(factor=80)
        self.mult_gain = ipytone.Gain(gain=0)
        self.clip = ipytone.WaveShaper(curve=[-1, 0, 1])
        self.noise1.chain(
            self.lp_filter,
            self.mult,
            self.mult_gain,
            self.clip,
            self.bp_filter,
            self.gain,
            self.panner,
        )
        self.noise2.connect(self.mult_gain.gain)
        
    @property
    def output(self):
        return self.panner
    
    def capture(self, delay, value=0.8):
        self.gain.gain.set_value_at_time(0, None)
        curve = np.array([0, 2, 0.4, 0.5, 0.25, 1.25, 0.6, 1.15, 0.35, 0.15, 0.001])
        self.gain.gain.set_value_curve_at_time(
            curve * value,
            f"+{delay}",
            14,
        
        )
        self.bp_filter.frequency.cancel_and_hold_at_time(None)
        self.bp_filter.frequency.set_value_at_time(
            333 + random.uniform(-100, 100), f"+{delay + 0.2}"
        )
        self.bp_filter.frequency.linear_ramp_to_value_at_time(
            233 + random.uniform(-50, 50), f"+{delay + 0.2 + 14}"
        )
        self.lp_filter.frequency.cancel_and_hold_at_time(None)
        self.lp_filter.frequency.set_value_at_time(33, f"+{delay + 0.2}")
        self.lp_filter.frequency.linear_ramp_to_value_at_time(
            0, f"+{delay + 0.2 + 14}"
        )
    
    def dispose(self):
        self.noise1.dispose()
        self.noise2.dispose()
        self.gain.dispose()
        self.lp_filter.dispose()
        self.bp_filter.dispose()
        self.panner.dispose()
        self.mult.dispose()
        self.mult_gain.dispose()
        self.clip.dispose()
    

In [ ]:
after_image = AfterImage()
after_image.output.connect(ipytone.get_destination())

In [ ]:
distance_delay = 0.29

for strike in strikes:
    strike.strike(distance_delay + random.uniform(0, 0.25))

after_image.capture(distance_delay, value=0.4)

## Rumbler

In [ ]:
class Rumbler:
    def __init__(self):
        self.noise1 = ipytone.Noise().start()
        self.noise2 = ipytone.Noise().start()
        self.gain1 = ipytone.Gain(gain=0)
        #self.gain2 = ipytone.Gain(gain=3)
        self.lp_filters = [
            ipytone.Filter(q=3) for _ in range(4)
        ]
        self.hp_filter = ipytone.Filter(type="highpass", frequency=300, q=8)
        
        self.panner = create_panner(
            random.uniform(-0.5, 0.5),
            random.uniform(-0.5, 0.5),
            random.uniform(-0.5, 0.5),
        )
        
        self._connect()
        
    def _connect(self):
        self.clip = ipytone.WaveShaper(curve=[0, 0, 1])
        self.clip2 = ipytone.WaveShaper(curve=[0.2, 0.2, 1])
        self.mult_gain = ipytone.Gain(gain=0)
        self.noise1.chain(
            self.lp_filters[0],
            self.clip,
            self.mult_gain,
            self.gain1,
            self.hp_filter,
            self.panner,
        )
        self.noise2.chain(
            self.lp_filters[1],
            self.clip2,
            self.mult_gain.gain
        )
    
    @property
    def output(self):
        return self.panner
    
    def rumble(self, delay, value=0.2):
        self.gain1.gain.set_value_at_time(0, None)
        curve = np.array([2.5, 0.15, 1.7, 0.12, 0.8, 0.25, 0.05, 0.0001])
        self.gain1.gain.set_value_curve_at_time(
            curve * value,
            f"+{delay}",
            9,
        
        )
    
        for filtr in self.lp_filters:
            filtr.frequency.cancel_and_hold_at_time(None)
            filtr.frequency.set_value_at_time(1000, f"+{delay}")
            filtr.frequency.linear_ramp_to_value_at_time(
            0, f"+{delay + 12}"
        )
    
    def dispose(self):
        self.noise1.dispose()
        self.noise2.dispose()
        self.gain1.dispose()
        #self.gain2.dispose()
        self.panner.dispose()
        for filtr in self.lp_filters:
            filtr.dispose()
        self.hp_filter.dispose()
        self.clip.dispose()
        self.clip2.dispose()
        self.mult_gain.dispose()
        

In [ ]:
rumbler = Rumbler()
rumbler.output.connect(ipytone.get_destination())

## Deepen

In [ ]:
class Deepen:
    def __init__(self):
        self.noise = ipytone.Noise().start()
        self.gain = ipytone.Gain(gain=0)
        self.hp_filter = ipytone.Filter(type="highpass", frequency=30, q=3)
        self.lp_filter1 = ipytone.Filter(frequency=60, q=3)
        self.lp_filter2 = ipytone.Filter(frequency=80, q=3)
        self.panner = create_panner(
            random.uniform(-0.4, 0.4),
            random.uniform(-0.2, 0.2),
            random.uniform(-0.5, 0.5),
        )
        
        self._connect()
    
    def _connect(self):
        self.mult = ipytone.Multiply(factor=3.5)
        self.clip = ipytone.WaveShaper(curve=[-1, 0, 1])
        self.noise.chain(
            self.lp_filter1,
            self.hp_filter,
            self.mult,
            self.clip,
            self.lp_filter2,
            self.gain,
            self.panner
        )
        
    @property
    def output(self):
        return self.panner
    
    def apply(self, delay, value=0.2):
        self.gain.gain.cancel_and_hold_at_time(None)
        self.gain.gain.set_value_at_time(0, f"+{delay}")
        curve = np.array([1, 6, 1.75, 5, 1.5, 4.15, 0.001])
        self.gain.gain.set_value_curve_at_time(curve * value, f"+{delay}", 16)
        
    def dispose(self):
        self.noise.dispose()
        self.gain.dispose()
        self.lp_filter1.dispose()
        self.lp_filter2.dispose()
        self.hp_filter.dispose()
        self.panner.dispose()
        self.mult.dispose()
        self.clip.dispose()
      

In [ ]:
deepen = Deepen()
deepen.output.connect(ipytone.get_destination())

In [ ]:
distance_delay = 0.29

for strike in strikes:
    strike.strike(distance_delay + random.uniform(0, 0.25))

after_image.capture(distance_delay, value=0.15)
deepen.apply(distance_delay + 0.5, value=0.1)
rumbler.rumble(distance_delay, value=1.5)

lp_filter.frequency.cancel_and_hold_at_time(None)
lp_filter.frequency.set_value_at_time(14000, f"+{distance_delay + 0.2}")
lp_filter.frequency.linear_ramp_to_value_at_time(
    0, f"+{distance_delay + 0.2 + 16}"
)

In [ ]:
ipytone.get_destination().volume.value = 4

In [ ]:
rumbler.dispose()
deepen.dispose()

In [ ]:
after_image.dispose()

In [ ]:
for strike in strikes:
    strike.dispose()


In [ ]:
reverb.dispose()
delay1.dispose()
delay2.dispose()
lp_filter.dispose()